# Getting Your Data From Yelp!

In order to make sure you are on track to completing the project, you will complete this workbook first. Below are steps that you need to take in order to make sure you have your data from yelp and are ready to analyze it. Your cohort lead will review this workbook with you the Wednesday before your project is due.    

# Part 1 - Understanding your data and question

You will be pulling data from the Yelp API to complete your analysis. The API, however, provides you with a lot of information that will not be pertinent to your analysis. You will pull data from the API and parse through it to keep only the data that you will need. In order to help you identify that information,look at the API documentation and understand what data the API will provide you. 

Identify which data fields you will want to keep for your analysis. 

https://www.yelp.com/developers/documentation/v3/get_started

# Part 2 - Create ETL pipeline for the business data from the API

## Details

Now that you know what data you need from the API, you want to write code that will execute an API call, parse those results and then insert the results into the DB.  

It is helpful to break this up into three different functions (*API call, parse results, and insert into DB*) and then you can write a function/script that pull the other three functions together. 

Let's first do this for the Business endpoint.

## Request

### Imports and Setup

In [39]:
import requests
import pandas as pd
import json
import csv
import numpy as np

with open(r'C:\Users\bmcca\.secret\yelp_api.json') as f:
    keys = json.load(f)

client_id = keys['id']
yelp_key = keys['key']

### ƒ: yelp_request

 - Params: search term (eg. "wineries); location; yelp_key variable (from Imports); and changing setting to print details

In [40]:
# def yelp_request(term, location, yelp_key, verbose=True):
#     '''Adapted from Yelp API Lab: https://github.com/BenJMcCarty/dsc-yelp-api-lab/tree/solution'''
    
#     url = 'https://api.yelp.com/v3/businesses/search'

#     headers = {
#             'Authorization': 'Bearer {}'.format(yelp_key),
#         }

#     url_params = {
#                     'term': term.replace(' ', '+'),
#                     'location': location.replace(' ', '+'),
#                     'limit': 50
#                 }
#     response = requests.get(url, headers=headers, params=url_params)
    
#     if verbose == True:
#         print(response)
#         print(type(response.text))
#         print(response.text[:1000])
        
#     return response.json()

### Sending the request and saving the response

- Un-comment the next line to run the response

In [41]:
# response = yelp_request('winery','Southern California', yelp_key)
# response.keys()

#### Saving/Loading as JSON for simplicity while iterating

In [42]:
# try:
#     with open(r'data\response.txt', 'w') as f:
#         json.dump(response, f)
# except IOError:
#     print("I/O error")

In [43]:
# with open('data/response.txt') as json_file:
#     data = json.load(json_file)

### Identifying and Exploring Keys

In [44]:
# Identify keys

print(response.keys())

NameError: name 'response' is not defined

#### Exploring the "Businesses" Key

In [ ]:
response['businesses']

In [ ]:
# Show first item w/in list of businesses

response['businesses'][0]

In [ ]:
response['businesses'][0]['categories'][0]['alias']

In [ ]:
response['businesses'][0]['categories'][0]['title']

#### Exploring the "Total" Key

In [ ]:
response['total']

# How many businesses are there in total for my request?

#### Exploring the "Region" Key

In [ ]:
response['region']

# From which geographical area will my results come?

## Parse

In [ ]:
response.keys()

### ƒ: parse_data

In [46]:
def parse_data(list_of_data):
    '''Adapted from Tyrell's code'''  

    # Create empty list to store results
    
    parsed_data = []
    
    # Loop through each business in the list of businesses
    # Add specific k:v pairs to a dictionary
    # These pairs will be used to build a DF afterwards
    
    for business in list_of_data:
        if 'price' not in business:
            business['price'] = np.nan
            
            # Verify that the "price" key is in the selected business dict
            
        details = {'name': business['name'],
                     'location': ' '.join(business['location']['display_address']),
                     'id': business['id'],
                     #'categories': business['categories'],
                     'alias': business['categories'][0]['alias'],
                     'title': business['categories'][0]['title'],
                     'rating': business['rating'],
                     'review_count': business['review_count'],
                     'price': business['price'],
                     'latitude': business['coordinates']['latitude'],
                     'longitude': business['coordinates']['longitude']
                    }

        # If the "price" key is missing, then skip adding that key
        # This avoids an error when getting the desired info
        
#         else:
#             details = {'name': business['name'],
#                          'location': business['location']['display_address'],
#                          'id': business['id'],
#                          #'categories': business['categories'],
#                          'alias': business['categories'][0]['alias'],
#                          'title': business['categories'][0]['title'],
#                          'rating': business['rating'],
#                          'review_count': business['review_count'],
#                          'latitude': business['coordinates']['latitude'],
#                          'longitude': business['coordinates']['longitude']
#                         }
        
        # Add the new dictionary to the previous list
        
        parsed_data.append(details)
    
    # Adjust the 'location' value to be a single string (not 2 str in a list)
    
#     for biz in parsed_data:
#         biz['location'] = ' '.join(biz['location'])
        
    # Create a DataFrame from the resulting list
    
    df_parsed_data = pd.DataFrame(parsed_data)
    
#     df_parsed_data.dropna(inplace=True)
    
    return df_parsed_data

In [ ]:
# parsed_results = parse_data(response['businesses'])
# parsed_results

In [ ]:
# # Identify totals of NaN values

# parsed_results.isnull().sum()

In [ ]:
# # Simple statistical exploration of quantitative values

# parsed_results.describe()

## Updating Requests for Pagination

### ƒ: yelp_request_offset

In [ ]:
def yelp_request_offset(term, location, yelp_key, offset=0, verbose=False):
    '''Adapted from Yelp API Lab: https://github.com/BenJMcCarty/dsc-yelp-api-lab/tree/solution'''
    
    url = 'https://api.yelp.com/v3/businesses/search'

    headers = {
            'Authorization': 'Bearer {}'.format(yelp_key),
        }

    url_params = {
                    'term': term.replace(' ', '+'),
                    'location': location.replace(' ', '+'),
                    'limit': 50,
                    'offset': offset
                        }
    
    response = requests.get(url, headers=headers, params=url_params)
    
    if verbose == True:
        print(response)
        print(type(response.text))
        print(response.text[:1000])
        
    return response.json()

#### Test 1

In [ ]:
# test1 = yelp_request_offset('winery', 'San Diego', yelp_key, offset=0, verbose=False)
# test1

In [ ]:
# test1.keys()

In [ ]:
# test1['total']

In [ ]:
# len(test1['businesses'])

#### Test 2

In [ ]:
# test2 = yelp_request_offset('winery', 'San Diego', yelp_key, offset=50, verbose=False)
# test2

#### Loop

In [ ]:
# results = yelp_request_offset('winery', 'San Diego', yelp_key, offset=0, verbose=False)

# num_pages = results['total']//50+1

# print(results['total'])
# print(len(results['businesses']))
# print(num_pages)

In [ ]:
# cur = 50
# parsed_results_dfs = []

# for num in range(num_pages):
#     results = yelp_request_offset('winery', 'San Diego', yelp_key, offset=cur, verbose=False)
#     parsed_results = parse_data(results['businesses'])
#     parsed_results_dfs.append(parsed_results)
#     cur += 50
    

In [ ]:
# parsed_results_dfs

In [ ]:
# df_concat = pd.concat(parsed_results_dfs, ignore_index=True)
# df_concat

## Save

### ƒ: df_save

Save parsed results to a .csv file

In [ ]:
# df.shape

In [ ]:
# df_concat.to_csv('data\wineries.csv')

In [ ]:
# df = pd.read_csv("data/wineries.csv", index_col=0)
# df

#### Old Stuff

In [ ]:
# from helper_funcs import *

In [ ]:


# # create a variable  to keep track of which result you are in. 
# cur = 0

# #set up a while loop to go through and grab the result 
# while cur < num and cur < 1000:
#     #set the offset parameter to be where you currently are in the results 

#     #make your API call with the new offset number
#     results = yelp_call_offset('winery', 'San Diego', yelp_key, offset=cur, verbose=False)
    
#     #after you get your results you can now use your function to parse those results
#     parsed_results = parse_results(results)
    
#     # use your function to insert your parsed results into the db
#     df_save(parsed_results)
#     #increment the counter by 50 to move on to the next results
#     cur += 50

## Edit and Condense

Goal: condense the details down to specific functions (perhaps one function?) to pull, clean, and save data.

While it will take some experimentation to write the functions above, once you get them working it will be best to put them in a `.py` file and then import the functions to use in a script 

# ƒ: GET FULL DATA

In [ ]:
# results = yelp_request_offset('winery', 'San Diego', yelp_key, offset=0, verbose=False)

# num_pages = results['total']//50+1

# print(results['total'])
# print(len(results['businesses']))
# print(num_pages)

In [ ]:
num_pages = 6

for i in range(num_pages):
    print(i)

In [55]:
def get_full_data(term, location, yelp_key, file_name = 'data/wineries_raw.csv'):
    '''Requests all results from Yelp API; saves as a .csv; and returns a DataFrame.'''
    
    blank_df = pd.DataFrame()
    blank_df.to_csv(file_name)
    
    # Process first request to Yelp API and calculate number of pages 
    results = yelp_request_offset(term, location, yelp_key, offset=0, 
                                  verbose=False)
    num_pages = results['total']//50+1
    
    # Print out confirmation feedback
    print(f'For {term} and {location}: ')
    print(f"    Total number of results: {results['total']}.")
    print(f'    Total number of pages: {num_pages}.')
    
    # Create offset for further results and create empty list
    offset = 0
    
    # Created .csv as container for results
#     parsed_results_dfs = []

    # Retrieves remaining pages
    for num in range(num_pages-1):
        try:
            # Process API request
            results = yelp_request_offset(term, location, yelp_key,
                                          offset=offset, verbose=False)
            
            # From results, take values from "Businesses" key and save
            parsed_results = parse_data(results['businesses'])
            
            
            # Removed following line due to change to saving to df/csv
            ## parsed_results_dfs.append(parsed_results)
            
            # H2: save results to df?
            # 1. Create DF from results
            # 2. Save resulting DF to .csv created above
            parsed_results.to_csv(file_name, mode='a', index = False)
            
            # Increase offset to move to next "page" of data
            offset += 50
            
        except:
            # If error, print where the error happens
            print(f'Error on page {num}.')
            # Then save the results so far to the .csv
            parsed_results.to_csv(file_name, mode='a', index = False)

#     # Concatenate DataFrames and save to .csv
#     df_concat = pd.concat(parsed_results_dfs, ignore_index=True)

    # Created new file at start - don't need at end
#     try:
#         df_concat.to_csv(file_name)
#         print(f'Saved to {file_name}.')
#     except:
#         print(f'Error, did not save.')
        
    return parsed_results

In [61]:
# Read saved data from .csv from above

df2 = pd.read_csv('data/wineries_raw.csv', header = 0)
df2

,name,location,id,alias,title,rating,review_count,price,latitude,longitude
0,The Winery Restaurant & Wine Bar,"4301 La Jolla Village Dr Ste 2040 San Diego, C...",76ADW8x8J_69qbtsc5F-2g,bars,Bars,4.0,492,$$,32.872428,-117.213775
1,Bernardo Winery,"13330 Paseo Del Verano Norte San Diego, CA 92128",DknnpiG1p4OoM1maFshzXA,winetastingroom,Wine Tasting Room,4.5,626,$$,33.032800,-117.046460
2,Baja Winery Tours,"4629 Cass St San Diego, CA 92109",vVaNDvLrCCE_Cw_DyPnBpA,winetours,Wine Tours,5.0,66,NaN,32.798916,-117.252111
3,Callaway Vineyard & Winery,"517 4th Ave Ste 101 San Diego, CA 92101",Cn2_bpTngghYW1ej4zreZg,winetastingroom,Wine Tasting Room,5.0,100,$$,32.710751,-117.160918
4,Négociant Winery,"2419 El Cajon Blvd San Diego, CA 92104",Cc1sQWRWgGyMCjzX2mmMQQ,winetastingroom,Wine Tasting Room,4.5,103,$$,32.754880,-117.138280
...,...,...,...,...,...,...,...,...,...,...
192,San Diego Limobuses,"3333 Midway Dr Ste 206 San Diego, CA 92110",SCaFGyzrTGTI6aQHhLxbgA,limos,Limos,3.0,46,NaN,32.750060,-117.211380
193,Alpine Discount Liquor,"2223 Alpine Blvd Alpine, CA 91901",#NAME?,beer_and_wine,"Beer, Wine & Spirits",3.5,3,$,32.835287,-116.765910
194,Village Wine & Spirits,"1552 Encinitas Blvd Encinitas, CA 92024",XkGnb-YxP5MK_ok1X011RA,beer_and_wine,"Beer, Wine & Spirits",4.0,24,$$,33.045896,-117.255584
195,The Destination Wedding Group,"Escondido, CA 92033",lJwxe_fjdt-e8xPrDq63fA,wedding_planning,Wedding Planning,5.0,18,NaN,33.123470,-117.086520


## Cleaning Data

## Identifying top 3 aliases

In [65]:
# Create new DF showing only alias and title columns
df2_alias = df2.loc[:,['alias', 'title']]

# Identify top 2 aliases 
df2_alias_count = df2_alias.groupby('alias').count().sort_values(['title'],\
                                                        ascending=False)[:2]
df2_alias_count.reset_index(inplace=True)

print(df2_alias_count)

# display them as a list
aliases_top_3 = df2_alias_count['alias'].tolist()
print(aliases_top_3)

# Note: initially tried top 3, but it returned distributors, not wineries

             alias  title
0         wineries     49
1  winetastingroom     33
['wineries', 'winetastingroom']


In [66]:
# Selecting rows based on condition

df3 = df2[df2['alias'].isin(aliases_top_3)]
df3

,name,location,id,alias,title,rating,review_count,price,latitude,longitude
1,Bernardo Winery,"13330 Paseo Del Verano Norte San Diego, CA 92128",DknnpiG1p4OoM1maFshzXA,winetastingroom,Wine Tasting Room,4.5,626,$$,33.032800,-117.046460
3,Callaway Vineyard & Winery,"517 4th Ave Ste 101 San Diego, CA 92101",Cn2_bpTngghYW1ej4zreZg,winetastingroom,Wine Tasting Room,5.0,100,$$,32.710751,-117.160918
4,Négociant Winery,"2419 El Cajon Blvd San Diego, CA 92104",Cc1sQWRWgGyMCjzX2mmMQQ,winetastingroom,Wine Tasting Room,4.5,103,$$,32.754880,-117.138280
5,San Pasqual Winery - Seaport Village,"805 W Harbor Dr San Diego, CA 92101",gMW1RvyLu90RSQAY9UrIHw,winetastingroom,Wine Tasting Room,4.5,138,$$,32.708732,-117.168195
6,Domaine Artefact Vineyard & Winery,"15404 Highland Valley Rd Escondido, CA 92025",WqVbxY77Ag96X90LultCUw,wineries,Wineries,5.0,96,$$,33.068170,-117.001600
...,...,...,...,...,...,...,...,...,...,...
140,Roll OutThe Barrell Charity Event by Meritage,"162 S Rancho Santa Fe Rd Encinitas, CA 92024",wyLm9fIoamN-VALcu3nUVg,wineries,Wineries,4.0,1,NaN,33.037121,-117.238654
145,Licores Kentucky,Calle Puerto y 3ra S/N Col. Centro 22000 Tijua...,B7gID-M2EsdpthrTcwTNYA,wineries,Wineries,5.0,1,NaN,32.534236,-117.034976
148,Barrica 9,Av. Revolución 1265 Col. Zona Centro 22000 Tij...,HxTqmzT4G43iAKXrB3pqQg,winetastingroom,Wine Tasting Room,4.5,7,$$,32.530430,-117.036500
165,"RL Liquid Assets, Inc","5909 Sea Lion Pl Ste G Carlsbad, CA 92010",#NAME?,wineries,Wineries,3.0,2,NaN,33.134743,-117.248093


In [67]:
df3.to_csv('data/wineries_filtered_alias.csv')

In [68]:
df_saved = pd.read_csv("data/wineries_filtered_alias.csv", index_col=0)
df_saved

,name,location,id,alias,title,rating,review_count,price,latitude,longitude
1,Bernardo Winery,"13330 Paseo Del Verano Norte San Diego, CA 92128",DknnpiG1p4OoM1maFshzXA,winetastingroom,Wine Tasting Room,4.5,626,$$,33.032800,-117.046460
3,Callaway Vineyard & Winery,"517 4th Ave Ste 101 San Diego, CA 92101",Cn2_bpTngghYW1ej4zreZg,winetastingroom,Wine Tasting Room,5.0,100,$$,32.710751,-117.160918
4,Négociant Winery,"2419 El Cajon Blvd San Diego, CA 92104",Cc1sQWRWgGyMCjzX2mmMQQ,winetastingroom,Wine Tasting Room,4.5,103,$$,32.754880,-117.138280
5,San Pasqual Winery - Seaport Village,"805 W Harbor Dr San Diego, CA 92101",gMW1RvyLu90RSQAY9UrIHw,winetastingroom,Wine Tasting Room,4.5,138,$$,32.708732,-117.168195
6,Domaine Artefact Vineyard & Winery,"15404 Highland Valley Rd Escondido, CA 92025",WqVbxY77Ag96X90LultCUw,wineries,Wineries,5.0,96,$$,33.068170,-117.001600
...,...,...,...,...,...,...,...,...,...,...
140,Roll OutThe Barrell Charity Event by Meritage,"162 S Rancho Santa Fe Rd Encinitas, CA 92024",wyLm9fIoamN-VALcu3nUVg,wineries,Wineries,4.0,1,NaN,33.037121,-117.238654
145,Licores Kentucky,Calle Puerto y 3ra S/N Col. Centro 22000 Tijua...,B7gID-M2EsdpthrTcwTNYA,wineries,Wineries,5.0,1,NaN,32.534236,-117.034976
148,Barrica 9,Av. Revolución 1265 Col. Zona Centro 22000 Tij...,HxTqmzT4G43iAKXrB3pqQg,winetastingroom,Wine Tasting Room,4.5,7,$$,32.530430,-117.036500
165,"RL Liquid Assets, Inc","5909 Sea Lion Pl Ste G Carlsbad, CA 92010",#NAME?,wineries,Wineries,3.0,2,NaN,33.134743,-117.248093


# Part 3 -  Create ETL pipeline for the restaurant review data from the API

You've done this for the Businesses, now you need to do this for reviews. You will follow the same process, but your functions will be specific to reviews. Above you have a model of the functions you will need to write, and how to pull them together in one script. For this part, you have the process below 

## Getting Business IDs

- In order to pull the reveiws, you will need the business ids. So your first step will be to get all of the business ids from your businesses csv. 

### Open file and slice ID

1. Open data/wineries.csv
2. Slice out the 'name' and 'id' columns for each row

In [41]:
df_saved = pd.read_csv("data/filtered_aliases.csv", index_col=0)
df_saved.reset_index(drop=True, inplace=True)
df_saved

,name,location,id,alias,title,rating,review_count,price,latitude,longitude
0,Bernardo Winery,"13330 Paseo Del Verano Norte San Diego, CA 92128",DknnpiG1p4OoM1maFshzXA,winetastingroom,Wine Tasting Room,4.5,626,$$,33.032800,-117.046460
1,Callaway Vineyard & Winery,"517 4th Ave Ste 101 San Diego, CA 92101",Cn2_bpTngghYW1ej4zreZg,winetastingroom,Wine Tasting Room,5.0,100,$$,32.710751,-117.160918
2,Négociant Winery,"2419 El Cajon Blvd San Diego, CA 92104",Cc1sQWRWgGyMCjzX2mmMQQ,winetastingroom,Wine Tasting Room,4.5,103,$$,32.754880,-117.138280
3,San Pasqual Winery - Seaport Village,"805 W Harbor Dr San Diego, CA 92101",gMW1RvyLu90RSQAY9UrIHw,winetastingroom,Wine Tasting Room,4.5,138,$$,32.708732,-117.168195
4,Domaine Artefact Vineyard & Winery,"15404 Highland Valley Rd Escondido, CA 92025",WqVbxY77Ag96X90LultCUw,wineries,Wineries,5.0,96,$$,33.068170,-117.001600
...,...,...,...,...,...,...,...,...,...,...
77,Roll OutThe Barrell Charity Event by Meritage,"162 S Rancho Santa Fe Rd Encinitas, CA 92024",wyLm9fIoamN-VALcu3nUVg,wineries,Wineries,4.0,1,NaN,33.037121,-117.238654
78,Licores Kentucky,Calle Puerto y 3ra S/N Col. Centro 22000 Tijua...,B7gID-M2EsdpthrTcwTNYA,wineries,Wineries,5.0,1,NaN,32.534236,-117.034976
79,Barrica 9,Av. Revolución 1265 Col. Zona Centro 22000 Tij...,HxTqmzT4G43iAKXrB3pqQg,winetastingroom,Wine Tasting Room,4.5,7,$$,32.530430,-117.036500
80,"RL Liquid Assets, Inc","5909 Sea Lion Pl Ste G Carlsbad, CA 92010",-STecUUsS69EMSE7PxwPwA,wineries,Wineries,3.0,2,NaN,33.134743,-117.248093


In [42]:
# Slice out the elements from the "name" column

df_saved_name = df_saved['name'].to_list()
df_saved_name

['Bernardo Winery',
 'Callaway Vineyard & Winery',
 'Négociant Winery',
 'San Pasqual Winery - Seaport Village',
 'Domaine Artefact Vineyard & Winery',
 'Carruth Cellars Wine Garden',
 'Blue Door Urban Winery',
 'Cordiano Winery',
 'Pali Wine Co',
 'FruitCraft - Fermentery & Distillery',
 'LJ Crafted Wines - Wines & Tastings',
 'Speckle Rock Vineyards',
 'San Pasqual Winery',
 'Altipiano Vineyard & Winery',
 'Gianni Buonomo Vintners',
 'Koi Zen Cellars',
 'La Mesa Wine Works',
 'Abnormal Wine Company',
 'Granite Lion Cellars',
 'Highland Valley Vineyards',
 '80 Sips Around the World',
 'Three Hills Winery',
 "Woof'n Rose Winery and Vineyard",
 "Bastian's Vineyards",
 'Vineyard Grant James',
 'Rustic Ridge Vineyards',
 'Espinosa Vineyards and Winery',
 'Charlie & Echo',
 "Rose's Tasting Room",
 'So Cal Wines',
 'San Pasqual Winery Tasting Room & Gallery',
 'Chuparosa Vineyards',
 'California Wine Line',
 'Principe di Tricase Winery',
 'Hatfield Creek Winery',
 'Record Family Wines',
 'D

In [43]:
df_saved_id = df_saved['id'].to_list()
df_saved_id

['DknnpiG1p4OoM1maFshzXA',
 'Cn2_bpTngghYW1ej4zreZg',
 'Cc1sQWRWgGyMCjzX2mmMQQ',
 'gMW1RvyLu90RSQAY9UrIHw',
 'WqVbxY77Ag96X90LultCUw',
 'yZp9FdMH6Dmn98mfNInFHw',
 'ElE6Nj7iz-tNV4ebV6Clew',
 'Ub2bJsi7lIOQ9TyIKdHaJw',
 'fyh566YXm5XJ3Ntv_GLghg',
 'sCET1pLdKNNPBQJyjPOkww',
 '7i8AdZoDySoEdT_uoMMzNA',
 'ALTc2EAwGkWfVL9gpuGdqw',
 'Phqm2QvJE0uEqpxMvdzH_g',
 'NKsyDqmKiNWT4HtkBh6AdA',
 'o0flINbzGa8LDlM9Mtbfwg',
 'Oic9__gID0APdrBIjP4Xjw',
 'YVMiPOAPsEd1fXxNmpjEMA',
 'ptA-I2Kk78jydAeJIYs2Cg',
 '1PS0mcUZkJPyG72pk3zwVg',
 '9qalQETuaZnFDgZrsEB7MA',
 'XH7njeu3FvhxoTqzHTTE5g',
 'Ye4Pr0ZbC8sGPwpWqR5wvQ',
 'f62YFfiriHapBhadXfMquA',
 'BGfXAtLC5sHBWOKlcb17KA',
 'gFPq8yqFRd9caMgkO5HvpQ',
 '97NwIhRGPhv1c4DU9DMpPA',
 'wLEvfL3dKvMMPD17-lpcGw',
 '9YDsJ5OL1hkx_vqwlJhi_Q',
 'VksLDK5jI-O53y0fFoUlQQ',
 '0gGOIJo3MUVpuy8hAepCGQ',
 'ia3GRsrEDexiodnpXe0D6w',
 'D9rtDihupzA39yBh0YcAXg',
 '5fkdGagUJDuc3axRWchzJQ',
 'RdMY32yM5MRIztG9gzuDKw',
 'V3kR-0Nb6ISBQ8ebXZJeVQ',
 '3mk6Tl3irp7x8baYoLRu9w',
 'Wjn9nDudZgJ5okiB8Cbfmg',
 

## Requesting Reviews

- Write a function that takes a business id and makes a call to the API for reviews.


### ƒ: yelp_id_offset

In [44]:
#Testing how to create URL properly for API request

biz_id = "DknnpiG1p4OoM1maFshzXA"
url = 'https://api.yelp.com/v3/businesses/' + biz_id + '/reviews'
url

'https://api.yelp.com/v3/businesses/DknnpiG1p4OoM1maFshzXA/reviews'

In [45]:
# f'https /{biz_id}/reviews

In [46]:
# for biz in biz_ids:
#     reviews = reviewcall(biz)
#     parsed_reviews = parse_reviews(reviews, biz)
    

In [47]:
def yelp_id_offset(biz_id, yelp_key, offset=0, verbose=True):
    '''Adapted from Yelp API Lab: https://github.com/BenJMcCarty/dsc-yelp-api-lab/tree/solution'''
    
    url = 'https://api.yelp.com/v3/businesses/'+ f'{biz_id}' + '/reviews'

    headers = {
            'Authorization': 'Bearer {}'.format(yelp_key),
        }

    url_params = {
#                     'id': biz_id.replace(' ', '+'),
                    'limit': 50,
                    'offset': offset
                        }
    
    response = requests.get(url, headers=headers, params=url_params)
    
    if verbose == True:
        print(response)
        print(type(response.text))
        print(response.text[:1000])
        
        
    return response.json()

#### Test ID Offset

In [49]:
# print(len(reviews1['reviews']))

3


##### Explore Reviews1 Keys

In [66]:
# reviews_list

In [68]:
# print(type(reviews_list['reviews']))

In [70]:
# reviews1['reviews'][0]

In [71]:
# reviews1['reviews'][0].keys()

In [72]:
# reviews1['reviews'][0]['user']

### ƒ: Parse_Reviews

In [97]:
def parse_reviews(list_of_reviews):
    '''Adapted from Tyrell's code'''  

    # Create empty list to store results
    
    parsed_reviews = []
    
    # Loop through each review in the list of reviews
    # Add specific k:v pairs to a dictionary
    # These pairs will be used to build a DF afterwards
    
#     print('Completed: create list ' + f'{parsed_reviews}')
    
    for review in list_of_reviews:
        details = {'Reviewer Name': review['user']['name'],
            'Review ID': review['id'],
            'Time Created': review['time_created'],
            'Review Rating': review['rating'],
            'Review Text': review['text']
            }
        
        # Add the new dictionary to the empty list
        
        parsed_reviews.append(details)
    
#     print('Completed: append list')
    
    # Create a DataFrame from the resulting list
    
    df_parsed_reviews = pd.DataFrame(parsed_reviews)

#     print('Created DF')
    
    return df_parsed_reviews

In [98]:
# df_parsed_reviews = parse_reviews(reviews1['reviews'])
# # df_parsed_reviews

### ƒ: Get_Full_Reviews

In [126]:
def get_reviews(biz_id, yelp_key,verbose=False):
    '''Requests all results from Yelp API; saves as a .csv; and returns a DataFrame.'''
    
    # Process first request to Yelp API and calculate number of pages 
    results = yelp_id_offset(biz_id, yelp_key, offset=0, verbose=False)
    num_pages = results['total']//50+1
    
    # Print out confirmation feedback
    if verbose == True:
        print(f'For {biz_id}: ')
        print(f"    Total number of results: {results['total']}.")
        print(f'    Total number of pages: {num_pages}.')
    
    # Create offset for further results and create empty list
    cur = len(results['reviews'])
    parsed_results_dfs = []

    # Retrieves remaining pages
    for num in range(1,num_pages):
        try:
            results = yelp_id_offset(biz_id, yelp_key, offset=cur, verbose=False)
            parsed_results = parse_reviews(results['reviews'])
            parsed_results['Business ID'] = biz_id
            parsed_results_dfs.append(parsed_results)
            cur += len(results['reviews'])
        except Exception as e:
            print(f'Error on page {num}.')
            print(e)

    # Concatenate DataFrames and save to .csv
    try:
        df_concat = pd.concat(parsed_results_dfs, ignore_index=True)
        df_concat.to_csv('data/business_ids.csv')
        return df_concat
    except Exception as e:
        print(f'Error, did not concat.')
        print(e)
#         return parsed_results_dfs


#     try:
#         df_concat.to_csv('data/business_ids.csv')
# #         print(f'Saved to data/business_ids.csv.')
#     except:
#         print(f'Error, did not save.')
        
#     return df_concat

In [120]:
biz_id

'Phqm2QvJE0uEqpxMvdzH_g'

In [121]:
# results = yelp_id_offset(biz_id, yelp_key, offset=0, verbose=False)
# num_pages = 2 #results['total']//50+1

# # Print out confirmation feedback
# print(f'For {biz_id}: ')
# print(f"    Total number of results: {results['total']}.")
# print(f'    Total number of pages: {num_pages}.')

# # Create offset for further results and create empty list
# cur = 50
# parsed_results_dfs = []

# # Retrieves remaining pages
# for num in range(num_pages-1):
#     try:
#         results = yelp_id_offset(biz_id, yelp_key, offset=cur, verbose=False)
#         parsed_results = parse_reviews(results['reviews'])
#         parsed_results_dfs.append(parsed_results)
#         cur += 50
#     except Exception as e:
#         print(f'Error on page {num}.')
#         print(e)

# # Concatenate DataFrames and save to .csv
# df_concat = pd.concat(parsed_results_dfs, ignore_index=True)


In [124]:
reviews_list = []

for biz_id in df_saved_id:
    review_result = get_reviews(biz_id, yelp_key)
    if review_result is not None:
        reviews_list.append(review_result)

reviews_list
    


For DknnpiG1p4OoM1maFshzXA: 
    Total number of results: 626.
    Total number of pages: 13.
For Cn2_bpTngghYW1ej4zreZg: 
    Total number of results: 100.
    Total number of pages: 3.
Error on page 1.
'reviews'
For Cc1sQWRWgGyMCjzX2mmMQQ: 
    Total number of results: 103.
    Total number of pages: 3.
For gMW1RvyLu90RSQAY9UrIHw: 
    Total number of results: 138.
    Total number of pages: 3.
For WqVbxY77Ag96X90LultCUw: 
    Total number of results: 96.
    Total number of pages: 2.
For yZp9FdMH6Dmn98mfNInFHw: 
    Total number of results: 195.
    Total number of pages: 4.
For ElE6Nj7iz-tNV4ebV6Clew: 
    Total number of results: 76.
    Total number of pages: 2.
For Ub2bJsi7lIOQ9TyIKdHaJw: 
    Total number of results: 547.
    Total number of pages: 11.
For fyh566YXm5XJ3Ntv_GLghg: 
    Total number of results: 173.
    Total number of pages: 4.
For sCET1pLdKNNPBQJyjPOkww: 
    Total number of results: 307.
    Total number of pages: 7.
For 7i8AdZoDySoEdT_uoMMzNA: 
    Total numb

For Tu52NWMO_dnv5v4iyuLykg: 
    Total number of results: 17.
    Total number of pages: 1.
Error, did not concat.
No objects to concatenate
For fT9ysnQvYXw-RUD4GNodEQ: 
    Total number of results: 1.
    Total number of pages: 1.
Error, did not concat.
No objects to concatenate
For Z9e6h-DPZeugrv7shpfl5w: 
    Total number of results: 2.
    Total number of pages: 1.
Error, did not concat.
No objects to concatenate
For C8W1fWjVOyA10rufYYfM6Q: 
    Total number of results: 1.
    Total number of pages: 1.
Error, did not concat.
No objects to concatenate
For JVMnIFpnAxRNxHrJKqxWQg: 
    Total number of results: 6.
    Total number of pages: 1.
Error, did not concat.
No objects to concatenate
For DM_xFbiqpOgnDBCk9wUwow: 
    Total number of results: 5.
    Total number of pages: 1.
Error, did not concat.
No objects to concatenate
For 4nFSeNcQny91tOID2Jfv8Q: 
    Total number of results: 26.
    Total number of pages: 1.
Error, did not concat.
No objects to concatenate
For KkhZZgGdLyLJMf

[   Reviewer Name               Review ID         Time Created  Review Rating  \
 0     Patrick T.  _aZ_3xAVPVJOVC_LlL0GeQ  2021-04-13 19:18:16              5   
 1      Bianca B.  _HFTJW4nRUe5lC8M1LLflA  2021-02-21 15:29:51              5   
 2   stephanie l.  AmyrN0N2SjWI_Uzy3Uq0jQ  2021-02-21 17:19:02              5   
 3     Patrick T.  _aZ_3xAVPVJOVC_LlL0GeQ  2021-04-13 19:18:16              5   
 4      Bianca B.  _HFTJW4nRUe5lC8M1LLflA  2021-02-21 15:29:51              5   
 5   stephanie l.  AmyrN0N2SjWI_Uzy3Uq0jQ  2021-02-21 17:19:02              5   
 6     Patrick T.  _aZ_3xAVPVJOVC_LlL0GeQ  2021-04-13 19:18:16              5   
 7      Bianca B.  _HFTJW4nRUe5lC8M1LLflA  2021-02-21 15:29:51              5   
 8   stephanie l.  AmyrN0N2SjWI_Uzy3Uq0jQ  2021-02-21 17:19:02              5   
 9     Patrick T.  _aZ_3xAVPVJOVC_LlL0GeQ  2021-04-13 19:18:16              5   
 10     Bianca B.  _HFTJW4nRUe5lC8M1LLflA  2021-02-21 15:29:51              5   
 11  stephanie l.  AmyrN0N2S

In [125]:
df_reviews = pd.concat(reviews_list, ignore_index=True)
df_reviews

,Reviewer Name,Review ID,Time Created,Review Rating,Review Text
0,Patrick T.,_aZ_3xAVPVJOVC_LlL0GeQ,2021-04-13 19:18:16,5,Debbie and Ronnie were wonderfully attentive t...
1,Bianca B.,_HFTJW4nRUe5lC8M1LLflA,2021-02-21 15:29:51,5,I haven't gone to the winery since pre covid; ...
2,stephanie l.,AmyrN0N2SjWI_Uzy3Uq0jQ,2021-02-21 17:19:02,5,I can't believe I never been here before! It's...
3,Patrick T.,_aZ_3xAVPVJOVC_LlL0GeQ,2021-04-13 19:18:16,5,Debbie and Ronnie were wonderfully attentive t...
4,Bianca B.,_HFTJW4nRUe5lC8M1LLflA,2021-02-21 15:29:51,5,I haven't gone to the winery since pre covid; ...
...,...,...,...,...,...
214,Grace P.,ZXanvmXHxB2FhbuHutQjhw,2021-04-03 02:26:48,1,"Okay. So, this place is a cool and interesting..."
215,Justin T.,0Wvt8Punl9ckYpuCUhEI-Q,2019-06-12 20:34:50,4,This place really has a lot of artwork & sculp...
216,Shannon T.,bBm-0DG0gZp93NHkZ8Racg,2019-08-27 17:14:22,5,"We moved to Ramona about 6 months ago, and hav..."
217,Grace P.,ZXanvmXHxB2FhbuHutQjhw,2021-04-03 02:26:48,1,"Okay. So, this place is a cool and interesting..."


## Parsing Reviews

- Write a function to parse out the relevant information from the reviews

### Adjust prior function

In [88]:
# def parse_data(list_of_data):
#     '''Adapted from Tyrell's code'''  


#     parsed_data = []
     
#     for business in list_of_data:
#         if 'price' in business:
#             details = {'name': business['name'],
#                          'location': business['location']['display_address'],
#                          'id': business['id'],
#                          #'categories': business['categories'],
#                          'alias': business['categories'][0]['alias'],
#                          'title': business['categories'][0]['title'],
#                          'rating': business['rating'],
#                          'review_count': business['review_count'],
#                          'price': business['price'],
#                          'latitude': business['coordinates']['latitude'],
#                          'longitude': business['coordinates']['longitude']
#                         }
#         else:
#             details = {'name': business['name'],
#                          'location': business['location']['display_address'],
#                          'id': business['id'],
#                          #'categories': business['categories'],
#                          'alias': business['categories'][0]['alias'],
#                          'title': business['categories'][0]['title'],
#                          'rating': business['rating'],
#                          'review_count': business['review_count'],
#                          'latitude': business['coordinates']['latitude'],
#                          'longitude': business['coordinates']['longitude']
#                         }
    
#         parsed_data.append(details)
    
#     for biz in parsed_data:
#         biz['location'] = ' '.join(biz['location'])
        
#     df_parsed_data = pd.DataFrame(parsed_data)
    
# #     df_parsed_data.dropna(inplace=True)
    
#     return df_parsed_data

## Saving Parsed Data

- Write a function to save the parse data into a csv file containing all of the reviews. 

### Adjust function to save to new file

In [87]:
# def get_full_data(term, location, yelp_key, file_name = 'data/wineries.csv'):
#     '''Requests all results from Yelp API; saves as a .csv; and returns a DataFrame.'''
    
#     # Process first request to Yelp API and calculate number of pages 
#     results = yelp_request_offset(term, location, yelp_key, offset=0, verbose=False)
#     num_pages = results['total']//50+1
    
#     # Print out confirmation feedback
#     print(f'For {term} and {location}: ')
#     print(f"    Total number of results: {results['total']}.")
#     print(f'    Total number of pages: {num_pages}.')
    
#     # Create offset for further results and create empty list
#     cur = 50
#     parsed_results_dfs = []

#     # Retrieves remaining pages
#     for num in range(num_pages-1):
#         try:
#             results = yelp_request_offset(term, location, yelp_key, offset=cur, verbose=False)
#             parsed_results = parse_data(results['businesses'])
#             parsed_results_dfs.append(parsed_results)
#             cur += 50
#         except:
#             print(f'Error on page {num}.')

#     # Concatenate DataFrames and save to .csv
#     df_concat = pd.concat(parsed_results_dfs, ignore_index=True)

#     try:
#         df_concat.to_csv(file_name)
#         print(f'Saved to {file_name}.')
#     except:
#         print(f'Error, did not save.')
        
#     return df_concat

## Combining Functions

- Combine the functions above into a single script  

# Part 4 -  Using python and pandas, write code to answer the questions below. 

**Reviews**

Which are the 5 most reviewed businesses in your dataset?

What is the highest rating received in your data set and how many businesses have that rating?

What percentage of businesses have a rating greater than or  4.5?

What percentage of businesses have a rating less than 3?

---

**Pricing**

What percentage of your businesses have a price label of one dollar sign? Two dollar signs? Three dollar signs? No dollar signs?

---

**Returing Reviews**

Return the text of the reviews for the most reviewed business. 

Find the highest rated business and return text of the most recent review. If multiple business have the same rating, select the business with the most reviews. 

Find the lowest rated business and return text of the most recent review.  If multiple business have the same rating, select the business with the least reviews. 

___

# Reference help

###  Pagination

Returning to the Yelp API, the [documentation](https://www.yelp.com/developers/documentation/v3/business_search) also provides us details regarding the API limits. These often include details about the number of requests a user is allowed to make within a specified time limit and the maximum number of results to be returned. In this case, we are told that any request has a maximum of 50 results per request and defaults to 20. Furthermore, any search will be limited to a total of 1000 results. To retrieve all 1000 of these results, we would have to page through the results piece by piece, retriving 50 at a time. Processes such as these are often refered to as pagination.

Now that you have an initial response, you can examine the contents of the json container. For example, you might start with ```response.json().keys()```. Here, you'll see a key for `'total'`, which tells you the full number of matching results given your query parameters. Write a loop (or ideally a function) which then makes successive API calls using the offset parameter to retrieve all of the results (or 5000 for a particularly large result set) for the original query. As you do this, be mindful of how you store the data. 

**Note: be mindful of the API rate limits. You can only make 5000 requests per day, and APIs can make requests too fast. Start prototyping small before running a loop that could be faulty. You can also use time.sleep(n) to add delays. For more details see https://www.yelp.com/developers/documentation/v3/rate_limiting.**

***Below is sample code that you can use to help you deal with the pagination parameter and bring all of the functions together.***


***Also, something might cause your code to break while it is running. You don't want to constantly repull the same data when this happens, so you should insert the data into the database as you call and parse it, not after you have all of the data***


In [ ]:
# create a variable  to keep track of which result you are in. 
cur = 0

#set up a while loop to go through and grab the result 
while cur < num and cur < 1000:
    #set the offset parameter to be where you currently are in the results 
    url_params['offset'] = cur
    #make your API call with the new offset number
    results = yelp_call(url_params, api_key)
    
    #after you get your results you can now use your function to parse those results
    parsed_results = parse_results(results)
    
    # use your function to insert your parsed results into the db
    db_insert(parsed_results)
    #increment the counter by 50 to move on to the next results
    cur += 20